In [1]:
# Plots for section 5.4 (hydrogen intersection with other sectors)

## Imports

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa
import math
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
from shapely.geometry import Point, LineString
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from datetime import date, datetime, time, timedelta

# imported own functions
from utils import market_values, market_values_by_time_index, nodal_balance, capacity, capacity_links, \
    capacity_storage_units, get_condense_sum, nodal_balance, generation, generation_links, \
    generation_storage_units, market_values_storage_units, market_values_links, time_stored_LIFO
from utils import generation_links_bus

# imported own definitions
from utils import carrier_colors, carrier_renaming, carrier_renaming_reverse, c_tags
from utils import resistive_heater, gas_boiler, heat_pump,water_tanks_charger, water_tanks_discharger, solar_thermal
from utils import c_el_gen_s, c_el_con_s, c_h2_gen, c_h2_con

# general variables
font1 = {'fontname': 'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/5.4_hydrogen_whyh2'
PLOT_DIR_add = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/additional_stuff'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger",
                  "solar thermal"]
markers = ["v", "^", "<", ">", "1", "2", "3", "4", "*", "+", "d", "o", "|", "s", "P", "p", "h"]

# new indices: cut off Fuel cell and SMR completely and SMR CC for exp
c_h2_gen_stst = ['H2 Electrolysis', 'SMR CC']
c_h2_gen_exp = ['H2 Electrolysis']
c_h2_gen = ['H2 Electrolysis', 'SMR CC']
c_h2_con = ['H2 liquefaction', 'Sabatier', 'Fischer-Tropsch']

In [3]:
# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

INFO:pypsa.io:Imported network elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [ ]:
# stst and exp dataframes
# spatial
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

# temporal
df_stst_ts = pd.read_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts = pd.read_pickle("../data/processed/df_exp_ts.pkl")

In [ ]:
# Notebook Functions


In [ ]:
#assert 0

## Balances

In [ ]:
# all buses
stst.buses.carrier.unique()
# Interesting: gas, H2, biogas, solid biomass, solid biomass for industry, gas for industry, H2 liquid, oil,

### all heat

In [ ]:
stst.buses.carrier.unique()[pd.Index(stst.buses.carrier.unique()).str.contains("heat")].tolist()

In [ ]:
# generation and consumption (all heat)

carriers = ['residential rural heat',
            'services rural heat',
            'residential urban decentral heat',
            'services urban decentral heat',
            'urban central heat']

fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(12, 4*4))

for n in [stst, exp]:

    nb = nodal_balance(n, carrier=carriers, time="2013", aggregate=['component', 'bus'], energy=True)
    # convert from MW to GW
    nb = nb.unstack(level=[1]) / 1000

    nb_pos = nb.sum()[nb.sum() > 0].sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_pos = nb_pos[(nb_pos / nb_pos.sum()) > 0.01]

    nb_neg = abs(nb.sum()[nb.sum() < 0]).sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_neg = nb_neg[(nb_neg / nb_neg.sum()) > 0.01]

    if n==stst:
        ax_gen=axs[0]
        ax_con=axs[2]
        title_gen=f"Generation: heat (STST)"
        title_con=f"Consumption: heat (STST)"

    elif n==exp:
        ax_gen=axs[1]
        ax_con=axs[3]
        title_gen=f"Generation: heat (EXP)"
        title_con=f"Consumption: heat (EXP)"

    # generation
    c = [carrier_colors[col] for col in nb_pos.index]
    percents = nb_pos.to_numpy() * 100 / nb_pos.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_pos.index, percents)]

    patches, texts = ax_gen.pie(nb_pos, colors=c, startangle=0, labels=labels)
    ax_gen.axis('equal')
    ax_gen.set_title(title_gen, pad=20, fontweight="bold")

    # consumption
    c = [carrier_colors[col] for col in nb_neg.index]
    percents = nb_neg.to_numpy() * 100 / nb_neg.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_neg.index, percents)]

    patches, texts = ax_con.pie(nb_neg, colors=c, startangle=0, labels=labels)
    ax_con.axis('equal')

    ax_con.set_title(title_con, pad=20, fontweight="bold")

fig.tight_layout(pad=4)
#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/heat_gen_con.png")

In [ ]:
# Overall heat generation

model = "EXP"

if model == "STST":
    n = stst

elif model == "EXP":
    n = exp

carrier = ['residential rural heat',
            'services rural heat',
            'residential urban decentral heat',
            'services urban decentral heat',
            'urban central heat']

nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
# convert from MWh to GWh
nb = nb.unstack(level=[1]) / 1000
# condense condense_groups
nb = get_condense_sum(nb, c1_groups, c1_groups_name)
# rename unhandy column names
nb.rename(columns=carrier_renaming, inplace=True)

nb[nb > 0].sum().sum() / 1e3

# STST: 3818 TWh
# EXP: 3407 TWh

### urban central heat

In [ ]:
# generation and consumption (urban central heat)

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [stst, exp]:

    carrier = ["urban central heat"]
    nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
    # convert from MW to GW
    nb = nb.unstack(level=[1]) / 1000

    nb_pos = nb.sum()[nb.sum() > 0].sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_pos = nb_pos[(nb_pos / nb_pos.sum()) > 0.01]

    nb_neg = abs(nb.sum()[nb.sum() < 0]).sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_neg = nb_neg[(nb_neg / nb_neg.sum()) > 0.01]

    if n==stst:
        ax_gen=axs[0, 0]
        ax_con=axs[0, 1]
        title_gen=f"Generation: {carrier} (STST)"
        title_con=f"Consumption: {carrier} (STST)"

    elif n==exp:
        ax_gen=axs[1, 0]
        ax_con=axs[1, 1]
        title_gen=f"Generation: {carrier} (EXP)"
        title_con=f"Consumption: {carrier} (EXP)"

    # generation
    c = [carrier_colors[col] for col in nb_pos.index]
    percents = nb_pos.to_numpy() * 100 / nb_pos.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_pos.index, percents)]

    patches, texts = ax_gen.pie(nb_pos, colors=c, startangle=0, labels=labels)
    ax_gen.axis('equal')
    ax_gen.set_title(title_gen, pad=20, fontweight="bold")

    # consumption
    c = [carrier_colors[col] for col in nb_neg.index]
    percents = nb_neg.to_numpy() * 100 / nb_neg.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_neg.index, percents)]

    patches, texts = ax_con.pie(nb_neg, colors=c, startangle=0, labels=labels)
    ax_con.axis('equal')

    ax_con.set_title(title_con, pad=20, fontweight="bold")

fig.tight_layout(pad=2)
#plt.close()
plt.show()

# Fischer Tropsch nebenprdukt Heat

### oil

In [ ]:
# generation and consumption (oil)
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [stst, exp]:

    carrier = ["oil"]
    nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
    # convert from MW to GW
    nb = nb.unstack(level=[1]) / 1000

    nb_pos = nb.sum()[nb.sum() > 0].sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_pos = nb_pos[(nb_pos / nb_pos.sum()) > 0.01]

    nb_neg = abs(nb.sum()[nb.sum() < 0]).sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_neg = nb_neg[(nb_neg / nb_neg.sum()) > 0.01]

    if n==stst:
        ax_gen=axs[0, 0]
        ax_con=axs[0, 1]
        title_gen=f"Generation: {carrier} (STST)"
        title_con=f"Consumption: {carrier} (STST)"

    elif n==exp:
        ax_gen=axs[1, 0]
        ax_con=axs[1, 1]
        title_gen=f"Generation: {carrier} (EXP)"
        title_con=f"Consumption: {carrier} (EXP)"

    # generation
    c = [carrier_colors[col] for col in nb_pos.index]
    percents = nb_pos.to_numpy() * 100 / nb_pos.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_pos.index, percents)]

    patches, texts = ax_gen.pie(nb_pos, startangle=0, colors=c, labels=labels)
    ax_gen.axis('equal')
    ax_gen.set_title(title_gen, pad=20, fontweight="bold")

    # consumption
    c = [carrier_colors[col] for col in nb_neg.index]
    percents = nb_neg.to_numpy() * 100 / nb_neg.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_neg.index, percents)]

    patches, texts = ax_con.pie(nb_neg,  startangle=0, colors=c, labels=labels)
    ax_con.axis('equal')

    ax_con.set_title(title_con, pad=20, fontweight="bold")

fig.tight_layout(pad=2)
#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/oil_gen_con.png")

### gas

In [ ]:
# generation and consumption (gas)
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [stst, exp]:

    carrier = ["gas"]
    nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
    # convert from MW to GW
    nb = nb.unstack(level=[1]) / 1000

    nb_pos = nb.sum()[nb.sum() > 0].sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_pos = nb_pos[(nb_pos / nb_pos.sum()) > 0.01]

    nb_neg = abs(nb.sum()[nb.sum() < 0]).sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_neg = nb_neg[(nb_neg / nb_neg.sum()) > 0.01]

    if n==stst:
        ax_gen=axs[0, 0]
        ax_con=axs[0, 1]
        title_gen=f"Generation: {carrier} (STST)"
        title_con=f"Consumption: {carrier} (STST)"

    elif n==exp:
        ax_gen=axs[1, 0]
        ax_con=axs[1, 1]
        title_gen=f"Generation: {carrier} (EXP)"
        title_con=f"Consumption: {carrier} (EXP)"

    # generation
    c = [carrier_colors[col] for col in nb_pos.index]
    percents = nb_pos.to_numpy() * 100 / nb_pos.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_pos.index, percents)]

    patches, texts = ax_gen.pie(nb_pos, startangle=0, colors=c, labels=labels)
    ax_gen.axis('equal')
    ax_gen.set_title(title_gen, pad=20, fontweight="bold")

    # consumption
    c = [carrier_colors[col] for col in nb_neg.index]
    percents = nb_neg.to_numpy() * 100 / nb_neg.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_neg.index, percents)]

    patches, texts = ax_con.pie(nb_neg,  startangle=0, colors=c, labels=labels)
    ax_con.axis('equal')

    ax_con.set_title(title_con, pad=20, fontweight="bold")

fig.tight_layout(pad=5)
#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/gas_gen_con.png")

In [ ]:
n = exp
carrier = ["gas"]
nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
# convert from MW to GW
nb = nb.unstack(level=[1]) / 1000

nb_pos = nb.sum()[nb.sum() > 0].sort_values(ascending=False)
# exclude all shares smaller than 1 %
nb_pos = nb_pos[(nb_pos / nb_pos.sum()) > 0.01]

nb_neg = abs(nb.sum()[nb.sum() < 0]).sort_values(ascending=False)
# exclude all shares smaller than 1 %
nb_neg = nb_neg[(nb_neg / nb_neg.sum()) > 0.01]

### testing carrier

In [ ]:
# generation and consumption
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [stst, exp]:

    carrier = ["solid biomass"]
    nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
    # convert from MW to GW
    nb = nb.unstack(level=[1]) / 1000

    nb_pos = nb.sum()[nb.sum() > 0].sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_pos = nb_pos[(nb_pos / nb_pos.sum()) > 0.01]

    nb_neg = abs(nb.sum()[nb.sum() < 0]).sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_neg = nb_neg[(nb_neg / nb_neg.sum()) > 0.01]

    if n==stst:
        ax_gen=axs[0, 0]
        ax_con=axs[0, 1]
        title_gen=f"Generation: {carrier} (STST)"
        title_con=f"Consumption: {carrier} (STST)"

    elif n==exp:
        ax_gen=axs[1, 0]
        ax_con=axs[1, 1]
        title_gen=f"Generation: {carrier} (EXP)"
        title_con=f"Consumption: {carrier} (EXP)"

    # generation
    #c = [carrier_colors[col] for col in nb_pos.index]
    percents = nb_pos.to_numpy() * 100 / nb_pos.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_pos.index, percents)]

    patches, texts = ax_gen.pie(nb_pos, startangle=0, labels=labels)
    ax_gen.axis('equal')
    ax_gen.set_title(title_gen, pad=20, fontweight="bold")

    # consumption
    #c = [carrier_colors[col] for col in nb_neg.index]
    percents = nb_neg.to_numpy() * 100 / nb_neg.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_neg.index, percents)]

    patches, texts = ax_con.pie(nb_neg,  startangle=0, labels=labels)
    ax_con.axis('equal')

    ax_con.set_title(title_con, pad=20, fontweight="bold")

fig.tight_layout(pad=2)
#plt.close()
plt.show()

### testing buses

In [ ]:
# norway indices
i_buses = df_stst_ons.index[df_stst_ons.index.str.contains("IE")]
country = "Ireland"

# generation and consumption
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [stst, exp]:

    carrier = ["AC", "low voltage"]
    nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component'], energy=True)

    nb = pd.DataFrame(nb)
    nb = nb.unstack(level=[1])
    nb.columns = nb.columns.droplevel(0)
    nb = nb[i_buses].sum(axis=1)
    nb = nb.unstack(level=[1])

    nb_pos = nb.sum()[nb.sum() > 0].sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_pos = nb_pos[(nb_pos / nb_pos.sum()) > 0.01]

    nb_neg = abs(nb.sum()[nb.sum() < 0]).sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_neg = nb_neg[(nb_neg / nb_neg.sum()) > 0.01]

    if n==stst:
        ax_gen=axs[0, 0]
        ax_con=axs[0, 1]
        title_gen=f"Generation: {carrier} (STST)"
        title_con=f"Consumption: {carrier} (STST)"

    elif n==exp:
        ax_gen=axs[1, 0]
        ax_con=axs[1, 1]
        title_gen=f"Generation: {carrier} (EXP)"
        title_con=f"Consumption: {carrier} (EXP)"

    # generation
    #c = [carrier_colors[col] for col in nb_pos.index]
    percents = nb_pos.to_numpy() * 100 / nb_pos.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_pos.index, percents)]

    patches, texts = ax_gen.pie(nb_pos, startangle=0, labels=labels)
    ax_gen.axis('equal')
    ax_gen.set_title(title_gen, pad=20, fontweight="bold")

    # consumption
    #c = [carrier_colors[col] for col in nb_neg.index]
    percents = nb_neg.to_numpy() * 100 / nb_neg.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_neg.index, percents)]

    patches, texts = ax_con.pie(nb_neg,  startangle=0, labels=labels)
    ax_con.axis('equal')

    ax_con.set_title(title_con, pad=20, fontweight="bold")

fig.tight_layout(pad=5)
fig.suptitle(country, fontweight="bold")
#plt.close()
plt.show()
#fig.savefig(f"{PLOT_DIR_add}/el_gen_con_{country}.png")

## Sabatier

In [ ]:
# gas production
# Sabatier: 563 TWh (STST), 29 TWh (EXP)
c = "Sabatier"

In [ ]:
df_stst_ons[f"{c}_gen_gas"]

In [ ]:
n = stst
n.links[n.links.carrier == "Sabatier"]

## Fischer-Tropsch


In [ ]:
c = "Fischer-Tropsch"
n = stst
carrier = ["oil"]
carrier = ['residential rural heat',
            'services rural heat',
            'residential urban decentral heat',
            'services urban decentral heat',
            'urban central heat']

nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
# convert from MW to GW
nb = nb.unstack(level=[1]) / 1000

nb_pos = nb.sum()[nb.sum() > 0].sort_values(ascending=False)
# exclude all shares smaller than 1 %
nb_pos = nb_pos[(nb_pos / nb_pos.sum()) > 0.01]

nb_neg = abs(nb.sum()[nb.sum() < 0]).sort_values(ascending=False)
# exclude all shares smaller than 1 %
nb_neg = nb_neg[(nb_neg / nb_neg.sum()) > 0.01]

In [ ]:
nb_pos / 1e3

# Oil production: 995 TWh (STST), 986 TWh (EXP)
# total heat production: 355 TWh (STST), 352 TWh (EXP)

## Loads

In [ ]:
c_loads = n.loads.carrier.unique()
df_loads = pd.DataFrame(index = c_loads, columns=["overall_load"])
df_loads.index[df_loads.index.str.contains("heat")]

In [ ]:
for c_l in c_loads:
    index = n.loads[n.loads.carrier == c_l].index
    df_loads.loc[c_l , "overall_load"] = n.loads_t.p.loc[: , index].sum().sum() / 1e6 * 3 # TWh

In [ ]:
df_loads.sort_values(by="overall_load", ascending=False)

In [ ]:
stst.loads_t.p.loc[: , stst.loads.carrier.isin(["electricity"])]

In [ ]:
# Fischer Tropsch loads
plt.plot(stst.loads_t.p.loc[: , stst.loads.carrier.isin(["kerosene for aviation", "naphtha for industry", "agriculture machinery oil"])].sum(axis=1))

## Additional stuff

In [ ]:
# Active power at bus0 (positive if branch is withdrawing power from bus0).

In [ ]:
stst.links[stst.links.carrier == "Sabatier"]

In [ ]:
stst.links_t.p2.loc[: , stst.links.carrier == "Sabatier"]

In [ ]:
stst.links[stst.links.carrier == "SMR CC"]

In [ ]:
stst.links_t.p2.loc[: , stst.links.carrier == "SMR CC"]

In [ ]:
stst.links_t.p3.loc[: , stst.links.carrier == "SMR CC"]

In [ ]:
stst.buses_t.marginal_price.loc[: , stst.buses.carrier == "gas"]

In [ ]:
plt.plot(stst.buses_t.p.loc[:, stst.buses.carrier == "co2 stored"].cumsum())

In [ ]:
stst.buses.carrier.unique()

In [ ]:
# p: active power at bus (positive if net production) (MW)

# GWh
n = stst
# indices are the same
i_co2_stored_stores = n.stores[n.stores.carrier == "co2 stored"].index

fig, axs = plt.subplots(nrows=2, figsize=(14, 8))

for n, ax in zip([stst, exp], axs):
    (-(n.stores_t.p[i_co2_stored_stores].sum(axis=1) / 1e3 * 3).resample("3h").sum()).plot(ax=ax,
                                                                                lw=0.7,
                                                                                color='turquoise',
                                                                                label="feed- in and feed out pattern")
    (-(n.stores_t.p[i_co2_stored_stores].sum(axis=1) / 1e3 * 3).resample("D").mean()).plot(ax=ax,
                                                                                lw=1,
                                                                                color='black',
                                                                               label="feed- in and feed out pattern (daily mean)")
    ax.set_ylabel("Active power ($GWh_{h2}$)")
    ax.set_xlabel("")
    ax.set_title("STST" if n == stst else "EXP", fontsize=16, **font1)
    ax.legend(loc="lower left")

fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(14, 8))

for n, ax in zip([stst, exp], axs):
    (n.stores_t.e[i_co2_stored_stores].sum(axis=1) / 1e6).resample("3h").sum().plot(ax=ax,
                                                                                lw=0.7,
                                                                                color='turquoise',
                                                                                label="storage level")
    (n.stores_t.e[i_co2_stored_stores].sum(axis=1) / 1e6).resample("D").mean().plot(ax=ax,
                                                                                lw=1,
                                                                                color='black',
                                                                               label="storage level (daily mean)")
    ax.set_ylabel("$TWh_{h2}$")
    ax.set_xlabel("")
    ax.set_title("STST" if n == stst else "EXP", fontsize=16, **font1)
    ax.legend(loc="lower left")

fig.tight_layout()

In [ ]:
-(n.stores_t.p[i_co2_stored_stores].sum(axis=1) / 1e6 * 3).sum() # restriction of 200 MtCO2 pro Jahr